# Variational optimization of quantum Hamiltonians with particle-conserving circuits

### For both noise-free and noisy simulations

In [42]:
# import necessary libraries and modules
from Ansatzes_Hamiltonians_class import AnsatzCircuit
from Ansatzes_Hamiltonians_class import XXZ, HCBH_ladder_gauged as HCBH
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import COBYLA, ADAM, SLSQP
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms.minimum_eigensolvers import VQE
from timeit import default_timer as timer 
from scipy.io import loadmat, savemat
import os
import numpy as np

In [43]:
# Set the parameters for the circuit and the Hamiltonian model

# fixed for all simulations 
num_params_per_gate = 2
num_trials = 20
num_opt_steps = 10000  # my default value
num_shots = 1024
include_noise = False

# model to run
spin_model = "XXZ"   # list = ["XXZ", "XY", "HCBH_nogauge", "HCBH_uniform", "HCBH_staggered"]
circ_type = "BWC"  # list = ["BWC", "LC"]
initial_ent_value = "no"  # list = ["no", "yes"]

In [44]:
## classical optimizer to update (classical) parameters
optimizer = COBYLA()

## Choose the estimator
def choose_estimator(include_noise):
    if (include_noise):
        from qiskit_aer.noise import NoiseModel
        from qiskit.providers.fake_provider import FakeVigo
        
        # fake providers contain data from real IBM Quantum devices stored in Qiskit Terra,
        # and are useful for extracting realistic noise models.
        device = FakeVigo()
        noise_model = NoiseModel.from_backend(device)
        coupling_map = device.configuration().coupling_map

        print(noise_model)

        # set the noisy estimator
        estimator = Estimator(
            options={
                "method" : "statevector",
                "noise_model" : noise_model,
                "seed" : 7, 
                "shots" : num_shots,
                "coupling_map" : coupling_map
                },
                )    
        print("Using the noisy estimaor")
    else:
        # noise-free estimator
        estimator = Estimator(options={"seed": 7, "shots": num_shots})

        print("Using the noiseless estimaor")

    return estimator

In [51]:
# import the Ansatz circuit

# define the quantum circuit
def build_circuit(num_particles, num_sites, gate_class="A", entangle_initial_state="no", circ_type='BWC', num_params_per_gate=2):

    global num_params
    
    qc = AnsatzCircuit(
        num_particles,
        num_sites,
        num_params_per_gate,
        gate_class,
        entangle_initial_state, 
        circ_type  
    )
    
    if (qc.dim_subspace < 10):
        print(f"Worker {os.getpid()} with gate {gate_class}: The quantum circuit: ")
        print(qc.draw())
    
    num_params = qc.num_parameters
    
    # decompose circuit before running
    qc_elem = qc.decompose().decompose() 

    return  qc_elem
    #return  qc

In [46]:
# Define the model Hamiltonian
def get_Hamiltonian(num_sites, spin_model):
    global ref_value 

    if spin_model == "XXZ":
        amp = 1
        H_obj = XXZ(num_sites, amp)                
    elif spin_model == "XY":        
        amp = 0
        H_obj = XXZ(num_sites, amp)   # XY = XXZ if ZZ term is turned off
    elif spin_model == "HCBH_nogauge":
        alpha_gauge = 0 
        H_obj = HCBH(num_sites, alpha_gauge)
    elif spin_model == "HCBH_uniform":
        alpha_gauge = 0.5 
        H_obj = HCBH(num_sites, alpha_gauge)
    elif spin_model == "HCBH_staggered":
        alpha_gauge = 0.3 
        H_obj = HCBH(num_sites, alpha_gauge)

    
    H = H_obj.Hamiltonian()
    H_op = SparsePauliOp.from_list(H)

    # get the reference ground state energy for small system sizes
    ref_value = H_obj.get_minimum_eigenvalue(H_op.to_matrix())
    #ref_value = -8.822948255619554   # @@@ ONLY FOR 3 PARTICLES ON 8 SITES
    print(f"\nExact ground state energy: {ref_value}")
    
    return H_op

## Optimization

In [47]:
## initial and/or input state

def check_for_file(spin_model, num_particles, num_sites, gate_class, entangle_initial_state, include_noise):    
    
    global filename, counts_trials, values_trials, rel_errs_trials, std_errs_trials, sim_data

    if include_noise:
        filename = spin_model + "_" + str(num_particles) + "_" + str(num_sites) + "_" + gate_class + "_" \
                        + entangle_initial_state + "_" + "ent" + "_" + circ_type + "_" + "noise"
    else:
        filename = spin_model + "_" + str(num_particles) + "_" + str(num_sites) + "_" + gate_class + "_" \
                        + entangle_initial_state + "_" + "ent" + "_" + circ_type + "_" + "gaussian"

    if os.path.exists(filename):    
        print(f"\nWorker {os.getpid()} with gate {gate_class}: File exists. Will load old data to continue simulation")

        sim_data = loadmat(filename)        

        # recall the output arrays         
        counts_trials = sim_data['counts_trials'][0]
        values_trials = sim_data['values_trials'][0]
        rel_errs_trials = sim_data['rel_errs_trials'][0]
        std_errs_trials = sim_data['rel_errs_trials'][0]

    else:
        # create a new file/sim
        print(f"\nWorker {os.getpid()} with gate {gate_class}: Starting a fresh calculation")

        # save the optimized energy values for all iterations and for the # of trials
        counts_trials = np.zeros([num_trials], dtype=object)
        values_trials = np.zeros([num_trials], dtype=object)
        rel_errs_trials = np.zeros([num_trials], dtype=object)
        std_errs_trials = np.zeros([num_trials], dtype=object)

        sim_data = {'num_particles' : num_particles,
                      'num_sites' : num_sites,
                      'gate_class' : gate_class,
                      'entangle_initial_state' : entangle_initial_state,                  
                      'num_trials' : num_trials                
                     }

In [48]:
## data preprocessing before plotting

def postprocess(counts_trials, values_trials, rel_errs_trials, num_opt_steps, num_trials):
    """ compute averages of energy expectation values and rel. error per iteration """     

    ## First, repackage data into an a 'proper' 2D array from a 1-D array of 1-D array objects
    counts_trials_mat = np.zeros((num_opt_steps, num_trials))
    values_trials_mat = np.zeros((num_opt_steps, num_trials))
    relerr_trials_mat = np.zeros((num_opt_steps, num_trials))
    stderr_trials_mat = np.zeros((num_opt_steps, num_trials))
    
    for i in range(num_trials):
        counts_trials_mat[0:num_opt_steps, i] = counts_trials[i]
        values_trials_mat[0:num_opt_steps, i] = values_trials[i]
        relerr_trials_mat[0:num_opt_steps, i] = rel_errs_trials[i]
        stderr_trials_mat[0:num_opt_steps, i] = std_errs_trials[i]

    ## compute averages 
    avg_energy_trials = np.zeros(num_opt_steps)
    avg_relerr_trials = np.zeros(num_opt_steps)
    avg_stderr_trials = np.zeros(num_opt_steps)
    
    for i in range(num_opt_steps):     
        num_trials_eff = np.count_nonzero(counts_trials_mat[i,:])   # number of trials effective
        if num_trials_eff != 0:
            avg_energy_trials[i] = sum(values_trials_mat[i, :])/num_trials_eff
            avg_relerr_trials[i] = sum(relerr_trials_mat[i, :])/num_trials_eff
            avg_stderr_trials[i] = sum(stderr_trials_mat[i, :])/num_trials_eff
    
    return avg_energy_trials, avg_relerr_trials, avg_stderr_trials

In [49]:
## Main loop cell

# define the callback function to store intermediate result during optimization
def store_intermediate_result(eval_count, parameters, mean, std):                          
        counts[eval_count-1] = eval_count
        values[eval_count-1] = mean    
        rel_err_val = np.abs((mean-ref_value)/ref_value)
        rel_errs[eval_count-1] = rel_err_val
        std_errs[eval_count-1] = std['variance']


# the main VQE loop
#def vqe_main(num_particles, num_sites, gate_class, entangle_initial_state, circ_type, ham_model):
def vqe_main(argin):    

        start_time = timer()        

        # extract the arguments
        num_particles = argin[0]
        num_sites = argin[1]
        gate_class = argin[2]
        entangle_initial_state = argin[3]
        circ_type = argin[4]
        ham_model = argin[5]
        num_opt_steps = argin[6]      
        include_noise = argin[7]

        # set number of classical optimization steps
        optimizer.__init__(maxiter=num_opt_steps)
        # choose the estimator
        estimator = choose_estimator(include_noise)
        
        # build circuit and get Hamiltonian        
        qc_elem = build_circuit(num_particles, num_sites, gate_class, entangle_initial_state, circ_type)
        H_op = get_Hamiltonian(num_sites, ham_model)
        check_for_file(ham_model, num_particles, num_sites, gate_class, entangle_initial_state, include_noise)

        # main algorithm loop        
        for i in range(num_trials): 
            rng = np.random.default_rng(seed=i+1)
            #starting_point = rng.uniform(-np.pi, np.pi, num_params)
            starting_point = rng.normal(0, 0.0546, num_params)  # @@@ USED FOR 8 SITES
            
            # define callback
            global counts, values, rel_errs, std_errs
            counts = np.zeros(num_opt_steps, dtype=int); 
            values = np.zeros(num_opt_steps); 
            rel_errs = np.zeros(num_opt_steps)
            std_errs = np.zeros(num_opt_steps)

            # instantiate and run VQE
            vqe = VQE(estimator, qc_elem, optimizer=optimizer, initial_point = starting_point, callback=store_intermediate_result)
            result = vqe.compute_minimum_eigenvalue(H_op)
            Energy_estimate = result.eigenvalue.real                        
            
            # save results for each trial
            counts_trials[i] = counts
            values_trials[i] = values
            rel_errs_trials[i] = rel_errs
            std_errs_trials[i] = std_errs
            if np.mod(i,5)==0: 
                print(f"Worker {os.getpid()} with gate {gate_class}: Done {i+1} trial samples")  
                sim_data.update({
                    'counts_trials'  : counts_trials, 
                    'values_trials'  : values_trials,  
                    'rel_errs_trials' : rel_errs_trials,                
                    'std_errs_trials' : std_errs_trials
                    })
                savemat(filename, sim_data)
        
        print(f"Worker {os.getpid()} with gate {gate_class}: Simulation's finished. Completed {i+1} trial samples.")                  

        # Compute average energy and average relative error
        print(f"\nWorker {os.getpid()} with gate {gate_class}: Post-processing: Computing averages.")
        avg_energy_trials, avg_relerr_trials, avg_stderr_trials = postprocess(counts_trials, values_trials, rel_errs_trials, num_opt_steps, num_trials)
        print(f'Worker {os.getpid()} with gate {gate_class}: Last avg. energy :', avg_energy_trials[num_opt_steps-1])
        print(f'Worker {os.getpid()} with gate {gate_class}: Last avg. rel. error :', avg_relerr_trials[num_opt_steps-1])        
        print(f'Worker {os.getpid()} with gate {gate_class}: Last avg. std. error :', avg_stderr_trials[num_opt_steps-1])        

        # compute simulation's total time
        end_time = timer()
        elapsed_time = (end_time - start_time)/3600
        print(f"\nWorker {os.getpid()} with gate {gate_class}:Total time taken for simulation = ", elapsed_time, " hours")

        # finally, save all simulation data to disk
        print(f"\r\nWorker {os.getpid()} with gate {gate_class}:=== Saving final data to disk === ")
        sim_data.update({
                'num_opt_steps' : num_opt_steps,
                'optimizer' : optimizer,
                'counts_trials'  : counts_trials, 
                'values_trials'  : values_trials,  
                'rel_errs_trials' : rel_errs_trials,
                'Exact GS energy' : ref_value,
                'elapsed_time' : elapsed_time,
                'avg_energy_trials' : avg_energy_trials, 
                'avg_relerr_trials' : avg_relerr_trials,
                'avg_stderr_trials' : avg_stderr_trials
                })
        savemat(filename, sim_data)
        
        end_time = timer()
        print(f"\rWorker {os.getpid()} with gate {gate_class}:Done...Elapsed time = ", (end_time - start_time)/3600, " hours")

In [9]:
# (num_particles, num_sites, gate_class, entangle_initial_state, circ_type, ham_model, num_opt_steps, include_noise)
job_list1 = [
    (2,4,"A","no", "BWC","XXZ", 1000, False),
    (2,4,"B","no", "BWC","XXZ", 1000, False),
    (2,4,"G","no", "BWC","XXZ", 1000, False)
]

job_list2 = [
    (3,6,"A","no", "BWC","XXZ", 3500, False),
    (3,6,"B","no", "BWC","XXZ", 3500, False),
    (3,6,"G","no", "BWC","XXZ", 3500, False)
]

job_list3 = [
    (4,8,"A","no", "BWC","XXZ", 12000, False),
    (4,8,"B","no", "BWC","XXZ", 12000, False),
    (4,8,"G","no", "BWC","XXZ", 12000, False)
]

In [10]:
# parallelizing the run
import multiprocess
from multiprocess import Process
import os

#if __name__ == '__main__':    

# run simulations for noiseless simulations

# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list1)    

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor


Worker 76798 with gate A: The quantum circuit: Worker 76799 with gate B: The quantum circuit: 

Worker 76800 with gate G: The quantum circuit: 
          ┌───────────────┐                 ┌───────────────┐                 
q_0: ─────┤0              ├─────────────────┤0              ├─────────────────
     ┌───┐│  B(θ[0],𝜙[0]) │┌───────────────┐│  B(θ[3],𝜙[3]) │┌───────────────┐
q_1: ┤ X ├┤1              ├┤0              ├┤1              ├┤0              ├
     └───┘├───────────────┤│  B(θ[2],𝜙[2]) │├───────────────┤│  B(θ[5],𝜙[5]) │
q_2: ─────┤0              ├┤1              ├┤0              ├┤1              ├
     ┌───┐│  B(θ[1],𝜙[1]) │└───────────────┘│  B(θ[4],𝜙[4]) │└───────────────┘
q_3: ┤ X ├┤1              ├─────────────────┤1              ├─────────────────
     └───┘└───────────────┘                 └───────────────┘                           ┌───────────────┐                 ┌──────────────

In [11]:
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list2)

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor



Exact ground state energy: -9.974308535551684

Worker 77035 with gate A: Starting a fresh calculation

Exact ground state energy: -9.974308535551684

Worker 77037 with gate G: Starting a fresh calculation
Exact ground state energy: -9.974308535551684


Worker 77036 with gate B: Starting a fresh calculation
Worker 77035 with gate A: Done 1 trial samples
Worker 77036 with gate B: Done 1 trial samples
Worker 77037 with gate G: Done 1 trial samples
Worker 77035 with gate A: Done 6 trial samples
Worker 77036 with gate B: Done 6 trial samples
Worker 77037 with gate G: Done 6 trial samples
Worker 77035 with gate A: Done 11 trial samples
Worker 77035 with gate A: Done 16 trial samples
Worker 77036 with gate B: Done 11 trial samples
Worker 77037 with gate G: Done 11 trial samples
Worker 77035 with gate A: Simulation's finished. Completed 20 trial samples.

Worker 77035 with gate A: Post-processing: Computing

In [12]:
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list3)

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor



Exact ground state energy: -13.49973039475164

Worker 77272 with gate A: Starting a fresh calculation

Exact ground state energy: -13.49973039475164

Worker 77274 with gate G: Starting a fresh calculation

Exact ground state energy: -13.49973039475164

Worker 77273 with gate B: Starting a fresh calculation
Worker 77272 with gate A: Done 1 trial samples
Worker 77273 with gate B: Done 1 trial samples
Worker 77274 with gate G: Done 1 trial samples
Worker 77272 with gate A: Done 6 trial samples
Worker 77273 with gate B: Done 6 trial samples
Worker 77272 with gate A: Done 11 trial samples
Worker 77272 with gate A: Done 16 trial samples
Worker 77274 with gate G: Done 6 trial samples
Worker 77273 with gate B: Done 11 trial samples
Worker 77272 with gate A: Simulation's finished. Completed 20 trial samples.

Worker 77272 with gate A: Post-processing: Computing averages.
Worker 77272 with gate A: Last avg. e

In [35]:
job_list4 = [
    (3,8,"A","no", "BWC","XY", 9500, False),
    (3,8,"B","no", "BWC","XY", 9500, False),
    (3,8,"G","no", "BWC","XY", 9500, False)
]

# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list4)

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor



Exact ground state energy: -8.822948255619554

Worker 72589 with gate A: Starting a fresh calculation

Exact ground state energy: -8.822948255619554

Worker 72590 with gate B: Starting a fresh calculation

Exact ground state energy: -8.822948255619554

Worker 72591 with gate G: Starting a fresh calculation
Worker 72589 with gate A: Done 1 trial samples
Worker 72590 with gate B: Done 1 trial samples
Worker 72591 with gate G: Done 1 trial samples
Worker 72589 with gate A: Done 6 trial samples
Worker 72590 with gate B: Done 6 trial samples
Worker 72589 with gate A: Done 11 trial samples
Worker 72591 with gate G: Done 6 trial samples
Worker 72589 with gate A: Done 16 trial samples
Worker 72590 with gate B: Done 11 trial samples
Worker 72589 with gate A: Simulation's finished. Completed 20 trial samples.

Worker 72589 with gate A: Post-processing: Computing averages.
Worker 72589 with gate A: Last avg. e

In [11]:
import multiprocess
from multiprocess import Process
import os

In [12]:
job_list5 = [
    (4,8,"A","no", "BWC","XY", 12000, False),
    (4,8,"B","no", "BWC","XY", 12000, False),
    (4,8,"G","no", "BWC","XY", 12000, False)
]

# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list5)

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor



Exact ground state energy: -9.517540966287271

Worker 43050 with gate A: Starting a fresh calculation

Exact ground state energy: -9.517540966287271

Worker 43052 with gate G: Starting a fresh calculation

Exact ground state energy: -9.517540966287271

Worker 43051 with gate B: Starting a fresh calculation
Worker 43050 with gate A: Done 1 trial samples
Worker 43051 with gate B: Done 1 trial samples
Worker 43052 with gate G: Done 1 trial samples
Worker 43050 with gate A: Done 6 trial samples
Worker 43051 with gate B: Done 6 trial samples
Worker 43050 with gate A: Done 11 trial samples
Worker 43050 with gate A: Done 16 trial samples
Worker 43052 with gate G: Done 6 trial samples
Worker 43051 with gate B: Done 11 trial samples
Worker 43050 with gate A: Simulation's finished. Completed 20 trial samples.

Worker 43050 with gate A: Post-processing: Computing averages.
Worker 43050 with gate A: Last avg. e

In [50]:
import multiprocess
from multiprocess import Process
import os

job_list6 = [
    (4,8,"A","no", "BWC","XXZ", 12000, False),
    (4,8,"B","no", "BWC","XXZ", 12000, False),
    (4,8,"G","no", "BWC","XXZ", 12000, False)
]

# for use with gaussian initialization.


# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list6)

Using the noiseless estimaorUsing the noiseless estimaorUsing the noiseless estimaor



Exact ground state energy: -13.49973039475164

Worker 50982 with gate G: Starting a fresh calculation

Exact ground state energy: -13.49973039475164
Exact ground state energy: -13.49973039475164


Worker 50984 with gate B: Starting a fresh calculation
Worker 50985 with gate A: Starting a fresh calculation

Worker 50985 with gate A: Done 1 trial samples
Worker 50984 with gate B: Done 1 trial samples
Worker 50982 with gate G: Done 1 trial samples
Worker 50985 with gate A: Done 6 trial samples
Worker 50984 with gate B: Done 6 trial samples
Worker 50985 with gate A: Done 11 trial samples
Worker 50985 with gate A: Done 16 trial samples
Worker 50982 with gate G: Done 6 trial samples
Worker 50984 with gate B: Done 11 trial samples
Worker 50985 with gate A: Simulation's finished. Completed 20 trial samples.

Worker 50985 with gate A: Post-processing: Computing averages.
Worker 50985 with gate A: Last avg. e